# Monte Carlo Fitting for Parameter Extraction

This section performs Monte Carlo mass fits to extract fitting parameters that will be used later for real data analysis. The Monte Carlo samples provide clean signal shapes without background contamination, allowing precise determination of signal model parameters.

## Analysis Strategy

The fitting procedure is divided into **three decay modes**:

1. **B⁺ → D⁰π⁺ (B2OC)**: Single-mode fitting to extract double Crystal Ball parameters for the main B⁺ signal peak around 5278 MeV
2. **B⁺ → J/ψK⁺ (B2CC)**: Single-mode fitting to extract double Crystal Ball parameters for the J/ψK⁺ signal 
3. **B⁺ → J/ψπ⁺ (J/ψπ)**: Single-mode fitting to extract parameters for the misidentified J/ψπ⁺ peak (~40 MeV higher than J/ψK⁺)

## Parameter Extraction Purpose

• **Shape Parameters**: Crystal Ball tail parameters (α, n), core resolutions (σ), and mean positions
• **PDF Components**: Double Crystal Ball fractions and relative normalizations
• **Fixed vs Floating**: Determine which parameters to fix/float in subsequent real data fits
• **Systematic Studies**: Evaluate parameter stability across different data-taking periods (blocks)

## Implementation

Each Monte Carlo sample is fitted independently using:
- **Signal Model**: Double Crystal Ball PDFs (accounting for detector resolution and radiative tails)
- **Background Model**: Minimal exponential component (for any residual combinatorial background)
- **Parameter Storage**: Results saved in ROOT TTrees for later retrieval in real data analysis

The extracted parameters will constrain the signal shapes when fitting real data, improving fit stability and reducing systematic uncertainties in yield measurements.

In [ ]:
import ROOT as r
import numpy as np
import matplotlib.pyplot as plt
import math
from ROOT import RooFit
from ROOT import TChain
from ROOT import RooStats
from datetime import datetime
from uncertainties import ufloat
from scipy.optimize import curve_fit
from pathlib import Path

# Define data directories for organized file management
DATA_CLEAN = Path("data/processed_clean_bp_p")     # Directory containing cleaned B+ momentum corrected files
MC_DATA = Path("data/monte_carlo")            # Directory containing processed MC files

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
print(r.gROOT.GetVersion())

%jsroot on

c = r.TCanvas()

## Monte Carlo mass fit, $B^+ \rightarrow \bar{D}^0\pi^+$ 


In [ ]:
from array import array                     # ROOT TTrees need C-style arrays
import re                   
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = [
    MC_DATA/"2024_MC_B2OC_B5.root", MC_DATA/"2024_MC_B2OC_B6.root", MC_DATA/"2024_MC_B2OC_B7.root", MC_DATA/"2024_MC_B2OC_B8.root",

]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 4900, 5950)   

mean = r.RooRealVar("mean", "mean", 5278.46, 5250, 5300)

alpha = r.RooRealVar("alpha", "alpha", 1.25015, 1, 2)
n = r.RooRealVar("n", "n", 2.40748, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 11, 1.0, 30)  
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

tau = r.RooRealVar("tau", "Decay constant", -0.00219122, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -2.2701, -20, -0.01)
n_2 = r.RooRealVar("n2", "n2", 2.47972, 0.05, 50)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 12.7184, 1.0, 20)   
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.65, 0.0, 1.0)

mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(False)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting block file {fname}")

    f = r.TFile.Open(fname, "UPDATE")         
    tree = f.Get("ST-b2oc")
    if not tree:
        print("ST-b2oc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.5, Nevt * 0.05, Nevt * 1)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.1, Nevt * 0.0, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = make_branch("status")
        br_covqual        = make_branch("covQual")
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background),r.RooFit.FillColor(r.kGreen + 3),r.RooFit.FillStyle(3001),r.RooFit.DrawOption("F"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.LineColor(r.kGreen+2), r.RooFit.LineStyle(r.kDashed))
 
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue)) , r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F")
    
    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = "B^{+} \\rightarrow \\bar{D}^{0}\\pi^{+}" 

    canvas_name = f"dpi_mc_mass_fit_block{match.group(1)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str}" if match else f"{fname} mass fit"

    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2)
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+3)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Save log-y version
    frame.SetMinimum(0.3)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()



▶ fitting block file 2024_MC_B2OC_DOWN_B7.root
entries in tree = 59763

=== fitted yields ===
nsig  = 59294 ± 246
nbkg  = 475 ± 36
ntot  = 59769 (derived)
0.7262774299871978
background fraction = 0.007953+/-0.000605
fit status = 0, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #1863 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4871.36
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3106 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4898.38
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4688 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4888.02
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4793 because Bp_DTF_OwnPV_MASS cannot accommodate the value 6096.33
[#1] INFO:DataHandling -- R

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 66.6395 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -324915.3285 Edm =        8870.63587 NCalls =     33
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 0.315893
Info in <Minuit2>: MnHesse Done after 94.4425 ms
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -324915.3285
  Edm           : 7350.98971
  Internal parameters:	[    -0.3155557491     0.2356918764      0.304692654     0.1388456842     -0.909951031   -0.05265590826      1.477141165]	
  Internal gradient  :	[     -1779.921638     -3681.926879      -883.282287     -4684.277411      8317.038908     -25384.80995      592.7171996]	
  Internal covariance matrix:
[[    0.046062002   -0.024919632   -0.060767527  0.00083586465  -0.0040843837  0.00091691214  0.00026121333]
 [   -0.024919632    0.01

In [ ]:

file_name = "2024_MC_B2OC_B7.root"   
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())


res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  ntot   =", res_tree.ntot,      "±", res_tree.ntot_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

c = f.Get("dpi_mc_mass_fit_block7_log")  #
if c:
    c.Draw()                       
else:
    print("canvas object not found; check its name in `f.ls()`")




--- top-level keys in the file -------------------------
  ST-b2oc : TTree
  fit_results : TTree
  dpi_mc_mass_fit_block7 : TCanvas
  dpi_mc_mass_fit_block7_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err
  status
  covQual

entries in the tree: 1

first entry snapshot:
  mean   = 5278.522580086098 ± 0.06494910988021729
  nsig   = 59294.08413221724 ± 246.0059488187544
  nbkg   = 475.32463891999157 ± 36.3804046650782
  ntot   = 59769.408771137234 ± 248.68144421691468
  frac_cb_2   = 0.7262774299871978


## Monte Carlo mass fit,  $B^+ \rightarrow J/\psi K^+$ 


In [ ]:
from array import array                     # ROOT TTrees need C-style arrays
import re
r.EnableImplicitMT()                        # enable multi-threading in RooFit

# 1) list of files
files = [
    MC_DATA/"2024_MC_B2CC_B5.root", MC_DATA/"2024_MC_B2CC_B6.root", MC_DATA/"2024_MC_B2CC_B7.root", MC_DATA/"2024_MC_B2CC_B8.root",
]

# 2) shape parameters
x     = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5050, 5550)    

mean = r.RooRealVar("mean", "mean", 5279.38, 5250, 5300)

tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

alpha = r.RooRealVar("alpha", "alpha", 1.59501, 0.1, 2)
n = r.RooRealVar("n", "n", 2.75673, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 7.2753, 5, 10)
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -1.29179, -1.5, -0.1)
n_2 = r.RooRealVar("n2", "n2", 5.29, 0.5, 10)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 8.21279, 5, 10)
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.5, 0.0, 1.0)


mean.setConstant(False)
alpha.setConstant(True)
n.setConstant(True)
cb_sigma.setConstant(False)
tau.setConstant(False)
alpha_2.setConstant(True)
n_2.setConstant(True)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# 3) loop over files (fills)
for fname in files:

    print(f"\n▶ fitting Block file {fname}")

    f = r.TFile.Open(fname, "UPDATE")          
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping");  f.Close();  continue

    # convert to RooDataSet
    data  = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt  = data.numEntries()
    print(f"entries in tree = {Nevt}")

    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 1)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.1, Nevt * 0.0, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    sig_u = ufloat(nsig.getVal(), nsig.getError()) 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # 4) store results in /fit_results TTree  (create once, then append)
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        # function to create branches
        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach them to the branches
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # 5) make & save the canvas (linear + log)
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background),r.RooFit.FillColor(r.kGreen + 3),r.RooFit.FillStyle(3001),r.RooFit.DrawOption("F"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.LineColor(r.kGreen+2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))

    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    fill_str = f"Fill {match.group(2)}" if match and match.group(2) else ""
    title_latex = "B^{+} \\rightarrow J/\psi K^{+}"

    canvas_name = f"jpsik_mc_mass_fit_block{match.group(1)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str}" if match else f"{fname} mass fit"

    # Draw frame and save linear-y canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)  # (x1, y1, x2, y2) 
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    # For background, add a dummy object for the fill
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen+2)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 
    
    frame.SetMinimum(50)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()


<>:219: SyntaxWarning: invalid escape sequence '\p'
<>:219: SyntaxWarning: invalid escape sequence '\p'
C:\Users\paolo\AppData\Local\Temp\ipykernel_24524\1166642859.py:219: SyntaxWarning: invalid escape sequence '\p'
  title_latex = "B^{+} \\rightarrow J/\psi K^{+}"



▶ fitting Block file 2024_MC_B2CC_UP_B8.root
entries in tree = 244253

=== fitted yields ===
nsig  = 235449 ± 489
nbkg  = 8806 ± 114
ntot  = 244255 (derived)
0.4335132529310381
background fraction = 0.036053+/-0.000454
fit status = 0, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #9 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5705.77
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #17 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5666.54
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #20 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5026.24
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #21 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5650.4
[#1] INFO:DataHandling -- RooTreeD

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 262.885 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -1697062.984 Edm =       96930.35125 NCalls =     37
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 0.71702
Info in <Minuit2>: MnHesse Done after 410.295 ms
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -1697062.984
  Edm           : 58128.56671
  Internal parameters:	[    -0.0900014567     0.2891274493                0     0.1761089065     -0.909951031    -0.6174371036      1.505265445]	
  Internal gradient  :	[      2754.216819      3196.662722       1303.06832      3304.863295      10804.12914     -251697.8479      1782.603091]	
  Internal covariance matrix:
[[    0.014580689   -0.033792313    0.033008262 -0.00042942648  0.00028299104 -6.7340854e-05 -7.9859223e-05]
 [   -0.033792313    0.07

In [ ]:

file_name = "2024_MC_B2CC_UP_B8.root"   
f = r.TFile.Open(file_name)

print("\n--- top-level keys in the file -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # optional: print the first row
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  nsig   =", res_tree.nsig,      "±", res_tree.nsig_err)
    print("  nbkg   =", res_tree.nbkg,      "±", res_tree.nbkg_err)
    print("  ntot   =", res_tree.ntot,      "±", res_tree.ntot_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)

c = f.Get("jpsik_mc_mass_fit_block8_log")  # linear-y canvas
if c:
    c.Draw()                       
else:
    print("canvas object not found; check its name in `f.ls()`")




--- top-level keys in the file -------------------------
  ST-b2cc : TTree
  ST-b2cc : TTree
  fit_results : TTree
  jpsik_mc_mass_fit_block8 : TCanvas
  jpsik_mc_mass_fit_block8_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err

entries in the tree: 1

first entry snapshot:
  mean   = 5279.374511136003 ± 0.02111946299783085
  nsig   = 235449.28721855782 ± 489.4580676682963
  nbkg   = 8806.166274551615 ± 113.52247429529962
  ntot   = 244255.45349310944 ± 502.4505469951341
  frac_cb_2   = 0.4335132529310381


## Monte Carlo mass fit,  $B^+ \rightarrow J/\psi \pi^+$ 

In [ ]:
import ROOT as r
from array import array
import re
from uncertainties import ufloat
r.EnableImplicitMT()

# Monte Carlo files for B+ → J/ψπ+ analysis
files = [
    MC_DATA/"2024_MC_B2CC_JPSIPI_B5.root", MC_DATA/"2024_MC_B2CC_JPSIPI_B6.root", MC_DATA/"2024_MC_B2CC_JPSIPI_B7.root", MC_DATA/"2024_MC_B2CC_JPSIPI_B8.root",
]

# Mass fitting parameters
x = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5100, 5700)    
mean = r.RooRealVar("mean", "mean", 5300, 5310, 5350)

# Background model
tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0)
background = r.RooExponential("background", "Exponential background", x, tau)

# Double Crystal Ball signal parameters
alpha = r.RooRealVar("alpha", "alpha", 1.59501, 0.1, 2)
n = r.RooRealVar("n", "n", 2.75673, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 7.2753, 5, 20)
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

alpha_2 = r.RooRealVar("alpha2", "alpha2", -1.29179, -1.5, -0.1)
n_2 = r.RooRealVar("n2", "n2", 5.29, 0.5, 10)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 8.21279, 5, 10)
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.5, 0.0, 1.0)

# Set parameter constraints
mean.setConstant(False)
alpha.setConstant(False)
n.setConstant(False)
cb_sigma.setConstant(False)
tau.setConstant(False)
alpha_2.setConstant(False)
n_2.setConstant(False)
cb_sigma_2.setConstant(False)
frac_cb_2.setConstant(False)

# Process each Monte Carlo file
for fname in files:

    print(f"\n▶ fitting Block file {fname}")

    f = r.TFile.Open(fname, "UPDATE")       
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping");  f.Close();  continue

    data = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt = data.numEntries()
    print(f"entries in tree = {Nevt}")

    # Yield parameters
    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 1)
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.1, Nevt * 0.0, Nevt * 0.95)
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball),
                                    r.RooArgList(frac_cb_2))

    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    
    sig_u = ufloat(nsig.getVal(), nsig.getError())
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # Store results in ROOT TTree
    f.cd()
    res_tree = f.Get("fit_results")
    
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")

        def make_branch(name):
            buf = array('d', [0.])
            res_tree.Branch(name, buf, f"{name}/D")
            return buf
        br_mean           = make_branch("mean")
        br_mean_e         = make_branch("mean_err")
        br_alpha          = make_branch("alpha1")
        br_alpha_e        = make_branch("alpha1_err")
        br_n              = make_branch("n1")
        br_n_e            = make_branch("n1_err")
        br_cb_sigma       = make_branch("cb_sigma1")
        br_cb_sigma_e     = make_branch("cb_sigma1_err")
        br_tau            = make_branch("tau")
        br_tau_e          = make_branch("tau_err")
        br_alpha_2        = make_branch("alpha2")
        br_alpha_2_e      = make_branch("alpha2_err")
        br_n_2            = make_branch("n2")
        br_n_2_e          = make_branch("n2_err")
        br_cb_sigma_2     = make_branch("cb_sigma2")
        br_cb_sigma_2_e   = make_branch("cb_sigma2_err")
        br_frac_cb_2      = make_branch("frac_cb_2")
        br_frac_cb_2_e    = make_branch("frac_cb_2_err")
        br_Nevt           = make_branch("Nevt")          
        br_nsig           = make_branch("nsig")
        br_nsig_e         = make_branch("nsig_err")
        br_nbkg           = make_branch("nbkg")
        br_nbkg_e         = make_branch("nbkg_err")
        br_ntot           = make_branch("ntot")
        br_ntot_e         = make_branch("ntot_err")
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])
    else:
        br_mean           = array('d', [0.])
        br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.])
        br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.])
        br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.])
        br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.])
        br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.])
        br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.])
        br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.])
        br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.])
        br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.])
        br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.])
        br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.])
        br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0])
        br_covqual        = array('i', [0])

        # Attach branches to tree
        res_tree.GetBranch("mean").SetAddress(br_mean)
        res_tree.GetBranch("mean_err").SetAddress(br_mean_e)
        res_tree.GetBranch("alpha1").SetAddress(br_alpha)
        res_tree.GetBranch("alpha1_err").SetAddress(br_alpha_e)
        res_tree.GetBranch("n1").SetAddress(br_n)
        res_tree.GetBranch("n1_err").SetAddress(br_n_e)
        res_tree.GetBranch("cb_sigma1").SetAddress(br_cb_sigma)
        res_tree.GetBranch("cb_sigma1_err").SetAddress(br_cb_sigma_e)
        res_tree.GetBranch("tau").SetAddress(br_tau)
        res_tree.GetBranch("tau_err").SetAddress(br_tau_e)
        res_tree.GetBranch("alpha2").SetAddress(br_alpha_2)
        res_tree.GetBranch("alpha2_err").SetAddress(br_alpha_2_e)
        res_tree.GetBranch("n2").SetAddress(br_n_2)
        res_tree.GetBranch("n2_err").SetAddress(br_n_2_e)
        res_tree.GetBranch("cb_sigma2").SetAddress(br_cb_sigma_2)
        res_tree.GetBranch("cb_sigma2_err").SetAddress(br_cb_sigma_2_e)
        res_tree.GetBranch("frac_cb_2").SetAddress(br_frac_cb_2)
        res_tree.GetBranch("frac_cb_2_err").SetAddress(br_frac_cb_2_e)
        res_tree.GetBranch("Nevt").SetAddress(br_Nevt)
        res_tree.GetBranch("nsig").SetAddress(br_nsig)
        res_tree.GetBranch("nsig_err").SetAddress(br_nsig_e)
        res_tree.GetBranch("nbkg").SetAddress(br_nbkg)
        res_tree.GetBranch("nbkg_err").SetAddress(br_nbkg_e)
        res_tree.GetBranch("ntot").SetAddress(br_ntot)
        res_tree.GetBranch("ntot_err").SetAddress(br_ntot_e)
        res_tree.GetBranch("status").SetAddress(br_status)
        res_tree.GetBranch("covQual").SetAddress(br_covqual)

    # Fill branch values
    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()
    res_tree.Write("", r.TObject.kOverwrite)

    # Generate mass fit plots
    frame = x.frame(r.RooFit.Title(f"{fname} mass fit"))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background),r.RooFit.FillColor(r.kGreen),r.RooFit.FillStyle(3001),r.RooFit.DrawOption("F"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.LineColor(r.kGreen+1), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))

    match = re.search(r'_B(\d+)(?:_F(\d+))?\.root$', fname)
    block_str = f"Block {match.group(1)}" if match else ""
    title_latex = r"B^{+} \rightarrow J/\psi \pi^{+}"

    canvas_name = f"jpsipi_mc_mass_fit_block{match.group(1)}" if match else f"mass_fit_{fname}"
    plot_title = f"{title_latex} Mass Fit, {block_str}" if match else f"{fname} mass fit"

    # Create and save canvas
    c = r.TCanvas(canvas_name, plot_title, 800, 600)
    frame.SetTitle(plot_title)
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    
    # Add legend
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    dummy_bg = r.TH1F("dummy_bg", "", 1, 0, 1)
    dummy_bg.SetFillColor(r.kGreen)
    dummy_bg.SetLineColor(r.kGreen+2)
    dummy_bg.SetFillStyle(3001)
    legend.AddEntry(dummy_bg, "Background", "f")
    legend.Draw()
    c.Write(canvas_name) 

    # Save log-scale version
    frame.SetMinimum(35)
    c.SetLogy()
    c.Write(canvas_name + "_log")

    f.Close()


▶ fitting Block file 2024_MC_B2CC_JPSIPI_DOWN_B7.root
entries in tree = 40554

=== fitted yields ===
nsig  = 25214 ± 999
nbkg  = 15340 ± 994
ntot  = 40554 (derived)
0.6147670325731056
background fraction = 0.378266+/-0.017864
fit status = 1, covQual = 3
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5041.11
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #5 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5003.36
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #9 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5033.95
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #13 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5062.84
[#1] INFO:DataHandling -- Ro

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 91.5471 ms
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -121698.7804 Edm =       30018752.26 NCalls =     45
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: NegativeG2LineSearch Done after 694.275 ms
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : -135525.531
  Edm           : 25257.49
  Internal parameters:	[     -1.057463502      1.117322355    -0.7706847527     -2.573561769                0    -0.6133668604   0.002991115571    -0.3140133768     -0.909951031    -0.6174371036....       1.505265445]	
  Internal gradient  :	[     -4214.257221      1006.354498      550.1912939     -103.3787719     -4151.911212     -1615.126678     -479.1388788      72.84806988     -22293.99898     -28990.54916....      

In [ ]:
# Results verification for B+ → J/ψπ+ Monte Carlo analysis
file_name = "2024_MC_B2CC_JPSIPI_B7.root"
f = r.TFile.Open(file_name)

print("\n--- ROOT file contents -------------------------")
for key in f.GetListOfKeys():
    print(" ", key.GetName(), ":", key.GetClassName())

# Examine fit results tree
res_tree = f.Get("fit_results")
if not res_tree:
    print("fit_results tree NOT found!")
else:
    print("\n--- fit_results branches ------------------------------")
    for br in res_tree.GetListOfBranches():
        print(" ", br.GetName())
    print("\nentries in the tree:", res_tree.GetEntries())

    # Display first entry results
    res_tree.GetEntry(0)
    print("\nfirst entry snapshot:")
    print("  mean   =", res_tree.mean,      "±", res_tree.mean_err)
    print("  frac_cb_2   =", res_tree.frac_cb_2)
    print("  n2   =", res_tree.n2)
    print("  alpha2   =", res_tree.alpha2)
    print("  cb_sigma2   =", res_tree.cb_sigma2)
    print("  cb_sigma1   =", res_tree.cb_sigma1)
    print("  n1   =", res_tree.n1)
    print("  alpha1   =", res_tree.alpha1)

# Display saved canvas
c = f.Get("jpsipi_mc_mass_fit_block7_log")
if c:
    c.Draw()
else:
    print("canvas object not found; check its name in `f.ls()`")


--- top-level keys in the file -------------------------
  ST-b2cc : TTree
  fit_results : TTree
  jpsipi_mc_mass_fit_block7 : TCanvas
  jpsipi_mc_mass_fit_block7_log : TCanvas

--- fit_results branches ------------------------------
  mean
  mean_err
  alpha1
  alpha1_err
  n1
  n1_err
  cb_sigma1
  cb_sigma1_err
  tau
  tau_err
  alpha2
  alpha2_err
  n2
  n2_err
  cb_sigma2
  cb_sigma2_err
  frac_cb_2
  frac_cb_2_err
  Nevt
  nsig
  nsig_err
  nbkg
  nbkg_err
  ntot
  ntot_err

entries in the tree: 1

first entry snapshot:
  mean   = 5319.143742575974 ± 0.26548517459286813
  frac_cb_2   = 0.6147670325731056
  n2   = 1.3659232231862677
  alpha2   = -0.6896234355003722
  cb_sigma2   = 9.752737283043485
  cb_sigma1   = 15.99305874871011
  n1   = 1.0851629919912373
  alpha1   = 1.8863420399399096


Error in <TList::Clear>: A list is accessing an object (0000028952527E90) already deleted (list name = TList)
